# Date Dimension Generator

Generate production-ready date dimension tables for your lakehouse with:
- US Federal & UK Bank holiday calendars
- Fiscal year alignment (any start month)
- ISO week numbering
- Weekend / business day flags
- 35+ pre-computed columns

## 1. Basic Date Dimension (US Calendar)

In [ ]:
from lakelogic.core.processor import DataProcessor
from lakelogic.core.dim_date import generate_date_dimension

df = generate_date_dimension(
    start_date="2024-01-01",
    end_date="2024-12-31",
    holiday_calendar="us",
    engine="polars",
)

print(f"Total rows: {df.height}")
print(f"Total columns: {len(df.columns)}")
print(f"Columns: {df.columns}")
df.head(10)

## 2. US Federal Holidays

In [ ]:
holidays = df.filter(df["is_holiday"] == True).select(
    "full_date", "day_name", "holiday_name"
)
holidays

## 3. Business Day Distribution

In [ ]:
total = df.height
weekends = df.filter(df["is_weekend"] == True).height
holidays_count = df.filter(df["is_holiday"] == True).height
business_days = df.filter(df["is_business_day"] == True).height

print(f"Total days:    {total}")
print(f"Weekends:      {weekends}")
print(f"Holidays:      {holidays_count}")
print(f"Business days: {business_days}")

## 4. Quarter Boundaries

In [ ]:
print("Quarter starts:")
quarter_starts = df.filter(df["is_quarter_start"] == True).select(
    "full_date", "quarter", "year_quarter"
)
print(quarter_starts)

print("\nQuarter ends:")
quarter_ends = df.filter(df["is_quarter_end"] == True).select(
    "full_date", "quarter", "year_quarter"
)
print(quarter_ends)

## 5. Fiscal Year — April Start (UK Government)

Many UK organisations use an April fiscal year:
- April → FY Q1
- January → FY Q4 (previous fiscal year)

In [ ]:
import polars as pl

df_apr = generate_date_dimension(
    start_date="2024-01-01",
    end_date="2025-03-31",
    fiscal_year_start_month=4,
    holiday_calendar="uk",
    engine="polars",
)

check_dates = [20240115, 20240415, 20240715, 20241015, 20250115]
for dk in check_dates:
    row = df_apr.filter(df_apr["date_key"] == dk)
    if row.height > 0:
        print(f"  {row['full_date'][0]}  →  FY{row['fiscal_year'][0]}  "
              f"Q{row['fiscal_quarter'][0]}  ({row['fiscal_year_quarter'][0]})")

print("\n  Key: Jan 2024 = FY2023-Q4, Apr 2024 = FY2024-Q1")

## 6. Fiscal Year — October Start (US Federal Government)

In [ ]:
df_oct = generate_date_dimension(
    start_date="2024-01-01",
    end_date="2025-09-30",
    fiscal_year_start_month=10,
    holiday_calendar="us",
    engine="polars",
)

for dk in [20240115, 20240415, 20241015, 20250115]:
    row = df_oct.filter(df_oct["date_key"] == dk)
    if row.height > 0:
        print(f"  {row['full_date'][0]}  →  FY{row['fiscal_year'][0]}  "
              f"Q{row['fiscal_quarter'][0]}  ({row['fiscal_year_quarter'][0]})")

print("\n  Key: Jan 2024 = FY2023-Q2, Oct 2024 = FY2024-Q1")

## 7. Custom Holidays

In [ ]:
df_custom = generate_date_dimension(
    start_date="2024-01-01",
    end_date="2024-12-31",
    holiday_calendar="us",
    custom_holidays={
        "2024-03-17": "St Patrick's Day",
        "2024-10-31": "Halloween",
        "2024-02-14": "Valentine's Day",
        "2024-04-01": "April Fool's Day",
    },
    engine="polars",
)

all_holidays = df_custom.filter(df_custom["is_holiday"] == True).select(
    "full_date", "holiday_name"
).sort("full_date")
all_holidays

## 8. UK Bank Holidays

In [ ]:
df_uk = generate_date_dimension(
    start_date="2024-01-01",
    end_date="2024-12-31",
    holiday_calendar="uk",
    engine="polars",
)

df_uk.filter(df_uk["is_holiday"] == True).select(
    "full_date", "day_name", "holiday_name"
)

## 9. DuckDB Output — Query Directly

In [ ]:
import duckdb
con = duckdb.connect()

table = generate_date_dimension(
    start_date="2020-01-01",
    end_date="2030-12-31",
    engine="duckdb",
    table_name="dim_date",
    connection=con,
)

print(f"Created table: {table}")
print(f"Total rows: {con.execute('SELECT count(*) FROM dim_date').fetchone()[0]:,}")

# Business days per quarter in 2024
con.execute("""
    SELECT year_quarter, count(*) as business_days
    FROM dim_date
    WHERE year = 2024 AND is_business_day = true
    GROUP BY year_quarter
    ORDER BY year_quarter
""").fetchdf()

In [ ]:
con.close()

## 10. Via DataProcessor (static method)

In [ ]:
df_via_proc = DataProcessor.generate_date_dimension(
    start_date="2025-01-01",
    end_date="2025-03-31",
    fiscal_year_start_month=4,
    engine="polars",
)

print(f"Generated {df_via_proc.height} rows")
print(f"Fiscal year for Jan 2025: FY{df_via_proc['fiscal_year'][0]}")
df_via_proc.head(5)